# QC on chromosome X: LBD (path + clin) vs. Controls (DementiaSeq, Wellderly, LNG) 

**Start date:** 07-27-2020

**Updated date:** 2024-01-10

**Author(s):** Ruth Chia

**Working directory on biowulf:** `/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM`

In [1]:
!pwd

/vf/users/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM


## What to do
1. Run chrX specific QC (variant level) separately on males and females; 
    - missingness by case-control (keep vars with  P > 1e-4)
    - missingness by haplotype case-control  (keep vars with  P > 1e-4)
    - keep vars with geno > 98% (i.e. --geno 0.02)
2. Then merge ; then ran another round of QC on merged data with additional parameters below   
3. Run XWAS


## QC parameters specific for chr X
1. Remove SNPs with significantly different MAF between male and female samples in the control group (use xwas --diff-x)
2. Remove SNPs with significantly different missingness rates between male and female controls (--test-missing ; code male = 1/"control" and female = 2/"case") or use --xwas --missdiff-x
3. Remove SNPs in the pseudoautosomal regions (PARs). 
4. Remove SNPs with HWE (female controls) midp < 1e-4



## What files I need
1. UnQC-ed chrX: `/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/neurod.freeze9.LBD.FTD.ALLcontrols4K.chrX.pgen/pvar/psam`

2. Make list of QC-ed LBD samples and controls from: `/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/CLEAN.UNRELATED/FILTERED.LBD.controls.UNRELATED_chrX.psam`


## Relevant papers
- `https://onlinelibrary.wiley.com/doi/10.1002/gepi.21782`
- `https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4567842/`
- `https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7132553/`
- `https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6781167/`
- `https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6646348/`
- `https://pubmed.ncbi.nlm.nih.gov/25479423/`


## Create unQC-ed chrX containing LBD samples + controls for chr X specific QC

In [ ]:
%%bash
module load plink/2.0-dev-20191128

DATA="/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020"

# Create list of samples to keep
awk '{print $1,$2}' OFS=" " $DATA/LBD/CLEAN.UNRELATED/FILTERED.LBD.controls.UNRELATED_chrX.psam > SampleList.LBD.controls.UNRELATED.FIDspaceIID.txt

# Make PAR region exclusion file for removal later
echo "23 10001 2781479 PAR1" > scripts/PAR_hg38_region.txt
echo "23 155701383 156030895 PAR2" >> scripts/PAR_hg38_region.txt

# Subset chr X
plink \
--pfile $DATA/neurod.freeze9.LBD.FTD.ALLcontrols4K.chrX \
--keep SampleList.LBD.controls.UNRELATED.FIDspaceIID.txt \
--set-hh-missing \
--keep-allele-order \
--make-bed \
--out neurod.LBD.controls.UNRELATED.chrX

## Run QC for chr X

In [ ]:
!cp -r ../Analysis.XWAS/scripts/ .

In [2]:
!cat ./scripts/QC_preimputation_v3_UNRELATED_wgs_chrX.sh

#! /bin/bash

basePlinkFILENAME=$1
email=$2

# Warning message to indicate argument requirement for script to run
if [ $# -eq 2 ]
then
    echo "QC_preimputation_v3_keepRelated_wgs.sh running"
    echo "This script should be executed in biowulf. If not, please terminate job."
    echo "This script uses plink2 that requires a processor which supports AVX2/Haswell instructions."
    echo "To be used for chrX QC on unrelated samples and EURO cohort i.e. will not perform ancestry check. Will assume samples are of one ancestry background."
    echo "Notification email will be sent when task is complete."
else
    echo "Need plink input file for chr X merged case/control data and email"
    echo "How to use: sh ../scripts/QC_preimputation_v3_UNRELATED_wgs_chrX.sh \$basePlinkFILENAME \$email"
    echo "Note: This script is written to be executed in biowulf."
    exit
fi

### How to use: sh QC_preimputation_v3_UNRELATED_wgs_chrX.sh $plinkINPUT $outputNAME $email
### How to run as a swarm job. 

In [ ]:
%%bash
sh ./scripts/QC_preimputation_v3_UNRELATED_wgs_chrX.sh neurod.LBD.controls.UNRELATED.chrX chiarp@mail.nih.gov

In [ ]:
!cat /data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS/scripts/QC_preimputation_v3_UNRELATED_wgs_chrX.sh

## ***NEW 06-02-2023:*** Run step() for sex-stratified cohorts using genetic PCs generated from autosome chromosomes

Note that the genetic PCs were generated and recorded in another notebook; refer to: `/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM/2024-01-10_Readme_ChromX_haplotypeblocks_estimation_PCA_calculations.ipynb`

In [ ]:
%%bash
module load R/3.5.2
R --vanilla --no-save

require(data.table)
require(tidyverse)

# read in file - male/female samples
data_0 <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.GLM/GLM.LBD.4Kcontrols/COVARIATES.freeze9.LBD.controls.UNRELATED.txt",header=T)
xwas <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS/COVARIATES.freeze9.LBD.controls.UNRELATED.exclGENDER.forXWAS.txt",header=T)
data <- subset(data_0, data_0$IID %in% xwas$IID) %>% filter(!is.na(CONSENSUS_AGE))

# test out squred age for covariate because the lambdas a quite inflated
## ref: https://stats.stackexchange.com/questions/19823/why-would-one-use-age-squared-as-a-covariate-in-a-genetic-association-study
data$AGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$AGExAGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$log_AGE <- log(data$CONSENSUS_AGE)
data$SEXxAGE <- data$GENDER * data$CONSENSUS_AGE
write.table(data,"COVARIATES.freeze9.LBD.controls.UNRELATED_clean.updated_covs.txt",sep="\t",quote=F,row.names=F,col.names=T)
dim(xwas)

table(data_0$AFFECTION_STATUS,data_0$SEX)

table(data$AFFECTION_STATUS,data$SEX)
data$CASE <- data$AFFECTION_STATUS - 1
table(data$CASE,data$GENDER)

# subset to females only
females <- data %>% filter(SEX == 2)
write.table(females,"COVARIATES.freeze9.LBD.controls.UNRELATED.females_clean.updated_covs.txt",sep="\t",quote=F,row.names=F,col.names=T)
table(females$CASE,females$GENDER)

# subset to males only
males <- data %>% filter(SEX == 1)
write.table(males,"COVARIATES.freeze9.LBD.controls.UNRELATED.males_clean.updated_covs.txt",sep="\t",quote=F,row.names=F,col.names=T)
table(males$CASE,males$GENDER)

# subset to female and male CASES only
cases <- data %>% filter(AFFECTION_STATUS == 2)
write.table(cases,"COVARIATES.freeze9.LBD.controls.UNRELATED.cases_clean.updated_covs.txt",sep="\t",quote=F,row.names=F,col.names=T)
table(cases$CASE,cases$GENDER)

# Run step() to determine which covariate is most important
# model using glm() and then use step() to determine the model that has the lowest AIC
# this tell us which covariates included in the model are important to use/for adjustment in analysis

## linear age ##
# for females only
model_females <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = females, family = "binomial"(link = "logit"))
summary(model_females)
step(model_females)

# for males only
model_males <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = males, family = "binomial"(link = "logit"))
summary(model_males)
step(model_males)

# for all male/female samples
model <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = data, family = "binomial"(link = "logit"))
summary(model)
step(model)

# for all cases samples
model <- glm(GENDER ~ CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = cases, family = "binomial"(link = "logit"))
summary(model)
step(model)

## ***NEW 12-08-2023:*** Run step() for sex-stratified cohorts using genetic PCs generated from X-chromosome

Note that the genetic PCs were generated and recorded in another notebook; refer to: `http://localhost:38407/lab/workspaces/auto-n/tree/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM/2024-01-10_Readme_ChromX_haplotypeblocks_estimation_PCA_calculations.ipynb`

In [ ]:
%%bash
module load R/4.3
R --vanilla --no-save

require(data.table)
require(tidyverse)

# read in file - male/female samples
data_0 <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.GLM/GLM.LBD.4Kcontrols/COVARIATES.freeze9.LBD.controls.UNRELATED.txt",header=T)
xwas <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS/COVARIATES.freeze9.LBD.controls.UNRELATED.exclGENDER.forXWAS.txt",header=T)

# remove autosomal PCs and replace with x-chromosome PCs
temp <- subset(data_0, data_0$IID %in% xwas$IID) %>% filter(!is.na(CONSENSUS_AGE)) %>%
        select(-PC1,-PC2,-PC3,-PC4,-PC5,-PC6,-PC7,-PC8,-PC9,-PC10)
pcs_x <- fread("X-PCA_redo/both/pcs_pruned_chrX_forPCA")
data <- merge(temp, pcs_x, by=c("FID","IID"))

# test out squred age for covariate because the lambdas a quite inflated
## ref: https://stats.stackexchange.com/questions/19823/why-would-one-use-age-squared-as-a-covariate-in-a-genetic-association-study
data$AGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$AGExAGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$log_AGE <- log(data$CONSENSUS_AGE)
data$SEXxAGE <- data$GENDER * data$CONSENSUS_AGE
write.table(data,"COVARIATES.freeze9.LBD.controls.UNRELATED_clean.updated_covs_using_x-PCs.txt",sep="\t",quote=F,row.names=F,col.names=T)
dim(xwas)

table(data_0$AFFECTION_STATUS,data_0$SEX)
table(data$AFFECTION_STATUS,data$SEX)
data$CASE <- data$AFFECTION_STATUS - 1
table(data$CASE,data$GENDER)

model <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = data, family = "binomial"(link = "logit"))
summary(model)
step(model)


# read in file - females only
data_0 <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.GLM/GLM.LBD.4Kcontrols/COVARIATES.freeze9.LBD.controls.UNRELATED.txt",header=T)
xwas <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS/COVARIATES.freeze9.LBD.controls.UNRELATED.exclGENDER.forXWAS.txt",header=T)

# remove autosomal PCs and replace with x-chromosome PCs
temp <- subset(data_0, data_0$IID %in% xwas$IID) %>% filter(!is.na(CONSENSUS_AGE)) %>%
        select(-PC1,-PC2,-PC3,-PC4,-PC5,-PC6,-PC7,-PC8,-PC9,-PC10)
pcs_x <- fread("X-PCA_redo/females_only/pcs_pruned_chrX_forPCA")
data <- merge(temp, pcs_x, by=c("FID","IID"))
head(data)

# test out squred age for covariate because the lambdas a quite inflated
## ref: https://stats.stackexchange.com/questions/19823/why-would-one-use-age-squared-as-a-covariate-in-a-genetic-association-study
data$AGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$AGExAGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$log_AGE <- log(data$CONSENSUS_AGE)
data$SEXxAGE <- data$GENDER * data$CONSENSUS_AGE
data$CASE <- data$AFFECTION_STATUS - 1
write.table(data,"COVARIATES.freeze9.LBD.controls.UNRELATED.females_clean.updated_covs_using_x-PCs.txt",sep="\t",quote=F,row.names=F,col.names=T)
table(data$CASE,data$GENDER)

model_females <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = data, family = "binomial"(link = "logit"))
summary(model_females)
step(model_females)


# read in file - males only
data_0 <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.GLM/GLM.LBD.4Kcontrols/COVARIATES.freeze9.LBD.controls.UNRELATED.txt",header=T)
xwas <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS/COVARIATES.freeze9.LBD.controls.UNRELATED.exclGENDER.forXWAS.txt",header=T)

# remove autosomal PCs and replace with x-chromosome PCs
temp <- subset(data_0, data_0$IID %in% xwas$IID) %>% filter(!is.na(CONSENSUS_AGE)) %>%
        select(-PC1,-PC2,-PC3,-PC4,-PC5,-PC6,-PC7,-PC8,-PC9,-PC10)
pcs_x <- fread("X-PCA_redo/males_only/pcs_pruned_chrX_forPCA")
data <- merge(temp, pcs_x, by=c("FID","IID"))

# test out squred age for covariate because the lambdas a quite inflated
## ref: https://stats.stackexchange.com/questions/19823/why-would-one-use-age-squared-as-a-covariate-in-a-genetic-association-study
data$AGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$AGExAGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$log_AGE <- log(data$CONSENSUS_AGE)
data$SEXxAGE <- data$GENDER * data$CONSENSUS_AGE
data$CASE <- data$AFFECTION_STATUS - 1
write.table(data,"COVARIATES.freeze9.LBD.controls.UNRELATED.males_clean.updated_covs_using_x-PCs.txt",sep="\t",quote=F,row.names=F,col.names=T)
table(data$CASE,data$GENDER)

model_males <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = data, family = "binomial"(link = "logit"))
summary(model_males)
step(model_males)

## ***NEW 12-19-2023:*** Run step() for sex-stratified cohorts using genetic PCs generated from merged autosomal and X-chromosome

Note that the genetic PCs were generated and recorded in another notebook; refer to: `http://localhost:38407/lab/workspaces/auto-n/tree/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM/2024-01-10_Readme_ChromX_haplotypeblocks_estimation_PCA_calculations.ipynb`

In [ ]:
%%bash
module load R/4.3
R --vanilla --no-save

require(data.table)
require(tidyverse)

# read in file - male/female samples
data_0 <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.GLM/GLM.LBD.4Kcontrols/COVARIATES.freeze9.LBD.controls.UNRELATED.txt",header=T)
xwas <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS/COVARIATES.freeze9.LBD.controls.UNRELATED.exclGENDER.forXWAS.txt",header=T)

# remove autosomal PCs and replace with x-chromosome PCs
temp <- subset(data_0, data_0$IID %in% xwas$IID) %>% filter(!is.na(CONSENSUS_AGE)) %>%
        select(-PC1,-PC2,-PC3,-PC4,-PC5,-PC6,-PC7,-PC8,-PC9,-PC10)
pcs_x <- fread("X_autosomal-PCA/both/pcs_FILTERED_ALLchr_X_forPCA")
data <- merge(temp, pcs_x, by=c("FID","IID"))

# test out squred age for covariate because the lambdas a quite inflated
## ref: https://stats.stackexchange.com/questions/19823/why-would-one-use-age-squared-as-a-covariate-in-a-genetic-association-study
data$AGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$AGExAGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$log_AGE <- log(data$CONSENSUS_AGE)
data$SEXxAGE <- data$GENDER * data$CONSENSUS_AGE
write.table(data,"COVARIATES.freeze9.LBD.controls.UNRELATED_clean.updated_covs_using_x-autosomal-PCs.txt",sep="\t",quote=F,row.names=F,col.names=T)
dim(xwas)

table(data_0$AFFECTION_STATUS,data_0$SEX)
table(data$AFFECTION_STATUS,data$SEX)
data$CASE <- data$AFFECTION_STATUS - 1
table(data$CASE,data$GENDER)

model <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = data, family = "binomial"(link = "logit"))
summary(model)
step(model)


# read in file - females only
data_0 <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.GLM/GLM.LBD.4Kcontrols/COVARIATES.freeze9.LBD.controls.UNRELATED.txt",header=T)
xwas <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS/COVARIATES.freeze9.LBD.controls.UNRELATED.exclGENDER.forXWAS.txt",header=T)

# remove autosomal PCs and replace with x-chromosome PCs
temp <- subset(data_0, data_0$IID %in% xwas$IID) %>% filter(!is.na(CONSENSUS_AGE)) %>%
        select(-PC1,-PC2,-PC3,-PC4,-PC5,-PC6,-PC7,-PC8,-PC9,-PC10)
pcs_x <- fread("X_autosomal-PCA/females_only/pcs_FILTERED_ALLchr_X_forPCA")
data <- merge(temp, pcs_x, by=c("FID","IID"))
head(data)

# test out squred age for covariate because the lambdas a quite inflated
## ref: https://stats.stackexchange.com/questions/19823/why-would-one-use-age-squared-as-a-covariate-in-a-genetic-association-study
data$AGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$AGExAGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$log_AGE <- log(data$CONSENSUS_AGE)
data$SEXxAGE <- data$GENDER * data$CONSENSUS_AGE
data$CASE <- data$AFFECTION_STATUS - 1
write.table(data,"COVARIATES.freeze9.LBD.controls.UNRELATED.females_clean.updated_covs_using_x-autosomal-PCs.txt",sep="\t",quote=F,row.names=F,col.names=T)
table(data$CASE,data$GENDER)

model_females <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = data, family = "binomial"(link = "logit"))
summary(model_females)
step(model_females)


# read in file - males only
data_0 <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.GLM/GLM.LBD.4Kcontrols/COVARIATES.freeze9.LBD.controls.UNRELATED.txt",header=T)
xwas <- fread("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS/COVARIATES.freeze9.LBD.controls.UNRELATED.exclGENDER.forXWAS.txt",header=T)

# remove autosomal PCs and replace with x-chromosome PCs
temp <- subset(data_0, data_0$IID %in% xwas$IID) %>% filter(!is.na(CONSENSUS_AGE)) %>%
        select(-PC1,-PC2,-PC3,-PC4,-PC5,-PC6,-PC7,-PC8,-PC9,-PC10)
pcs_x <- fread("X_autosomal-PCA/males_only/pcs_FILTERED_ALLchr_X_forPCA")
data <- merge(temp, pcs_x, by=c("FID","IID"))

# test out squred age for covariate because the lambdas a quite inflated
## ref: https://stats.stackexchange.com/questions/19823/why-would-one-use-age-squared-as-a-covariate-in-a-genetic-association-study
data$AGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$AGExAGExAGE <- data$CONSENSUS_AGE * data$CONSENSUS_AGE * data$CONSENSUS_AGE
data$log_AGE <- log(data$CONSENSUS_AGE)
data$SEXxAGE <- data$GENDER * data$CONSENSUS_AGE
data$CASE <- data$AFFECTION_STATUS - 1
write.table(data,"COVARIATES.freeze9.LBD.controls.UNRELATED.males_clean.updated_covs_using_x-autosomal-PCs.txt",sep="\t",quote=F,row.names=F,col.names=T)
table(data$CASE,data$GENDER)

model_males <- glm(CASE ~ GENDER + CONSENSUS_AGE + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 , data = data, family = "binomial"(link = "logit"))
summary(model_males)
step(model_males)